Load Video

1.   Load from Google Drive in Colab
2.   Make directory



In [1]:
# # Google Colab import files in google drive
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools 

# !add-apt-repository -y ppa:alessandro-strada/ppa
  
# !apt-get update -qq

# !apt-get -y install -qq google-drive-ocamlfuse fuse 

# from google.colab import auth
# auth.authenticate_user()

# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()

# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# !mkdir -p drive
# !google-drive-ocamlfuse drive
import os
if not os.path.exists("drive"):
    print("no drive")
    from google.colab import drive
    drive.mount('/content/drive')

In [2]:
# Check the dirctory
!dir "drive/My Drive/FaceClusterer/"

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: C0A1-30B9

 C:\Users\Joonho Wohn\Documents\Dev\FaceClusterer\doomsheart\drive\My Drive\FaceClusterer 디렉터리

2018-11-09  오후 01:44    <DIR>          .
2018-11-09  오후 01:44    <DIR>          ..
2018-11-09  오후 02:06    <DIR>          res
2018-11-09  오후 02:08    <DIR>          result
               0개 파일                   0 바이트
               4개 디렉터리  73,381,949,440 바이트 남음


In [4]:
!pip install tqdm
!pip install dlib
!pip install opencv-python
!pip install opencv-contrib-python
!pip install --upgrade imutils
!pip install cmake
!pip install dlib
!pip install scipy scikit-learn 

from tqdm import tqdm
import cv2 as cv
import numpy as np
import os
import datetime
import dlib
from matplotlib import pyplot as plt
%matplotlib inline
import shutil

BASE_DIR = 'drive/My Drive/FaceClusterer/'
RES_DIR = 'drive/My Drive/FaceClusterer/res/'

VIDEO_DIR = RES_DIR + 'vid/'
RESULT_DIR = 'drive/My Drive/FaceClusterer/result/'
# for unique
# PROGRAM_START_TIME = datetime.datetime.now().strftime('%Y%m%d_%H-%M-%S')
# PROGRAM_START_TIME = datetime.datetime.now().strftime('%Y%m%d')
PROGRAM_START_TIME = '20180827'


def make_directory(_directory_name):
    if not os.path.exists(_directory_name):
        os.makedirs(_directory_name)
        
def get_area_of_frame_face_recognition(img, face_cascade):
    grayed_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # def detectMultiScale(self, image, scaleFactor=None, minNeighbors=None, flags=None, minSize=None, maxSize=None)
    face_area = face_cascade.detectMultiScale(image=grayed_img,scaleFactor=1.3,minNeighbors=5)
    return face_area
  
def get_frame(video_capture, frame):
    video_capture.set(cv.CAP_PROP_POS_FRAMES, frame)
    ret, img = video_capture.read()
    return img, frame, ret

def save_cropped_img(img, faces, frame):
    i = 0
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            iter = 0.2         
            while True:
                try:
                    face_img = img[int(y - h * iter): int(y + (1 + iter) * h),
                                   int(x - w * iter): int(x + (1 + iter) * w)]
                    if int(y - h * iter) < 0 or int(x - w * iter) < 0:
                        raise ValueError('range is false')
                    break
                except:
                    iter = iter * 0.8
                  #             cv.imwrite(SAVE_CROPPED_FACE_DIR + "/" + str(frame) + "_" + str(i) + ".jpg", img[y:y + h, x: x + w])
            try:
                cv.imwrite(SAVE_CROPPED_FACE_DIR + "/" + str(frame) + "_" + str(i) + ".jpg", cv.resize(face_img, (96, 96), interpolation=cv.INTER_AREA))
            except:
                print(int(y - h * iter))
                print(int(x - w * iter))
            i += 1
            
            
# this function show image and quit when press q or end
def show_img(img, faces):
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            cv.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 1)
    cv.imshow('hello', img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        return False
    else:
        return True
      
def ORB():
    img = cv.imread(SAVE_CROPPED_FACE_DIR + '/130_0.jpg')
    imgray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img2 = None
    orb = cv.ORB_create()
    kp, des = orb.detectAndCompute(img, None)
#     img2 = cv.drawKeypoints(img, kp, img2, (0, 0, 255), flags=0)
#     plt.imshow(img2)
    return kp, des 

def HarrisCorner():
    img = cv.imread(SAVE_CROPPED_FACE_DIR + '/130_0.jpg')
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    corners = cv.cornerHarris(img_gray, 2, 3, 0.04)
#     plt.figure(figsize=(6, 6))
#     plt.imshow(corners, cmap='gray')
    return coners




Requirement already up-to-date: imutils in c:\users\joonho wohn\appdata\local\programs\python\python36\lib\site-packages (0.5.1)


In [5]:
VIDEO_FILE_NAME = 'PowerUp.mp4'
SAVE_CROPPED_FACE_DIR = RESULT_DIR + 'Cropped_imgs/' + PROGRAM_START_TIME + '-' + VIDEO_FILE_NAME.split('.')[0]
make_directory(SAVE_CROPPED_FACE_DIR)


m_video_capture = cv.VideoCapture(VIDEO_DIR + VIDEO_FILE_NAME)

CASCADE_DIR = RES_DIR + 'lbpcascade_frontalface_improved.xml'
m_face_cascade = cv.CascadeClassifier(CASCADE_DIR)

TOTAL_FRAME = m_video_capture.get(cv.CAP_PROP_FRAME_COUNT)
PROGRAM_START_TIME = '20181108'
predictor_path = RES_DIR + "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
def dlib_face_chip(img, frame_num):
    dets = detector(img, 1)
    if len(dets) == 0:
        return;
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(img, detection))
    images = dlib.get_face_chips(img, faces, size=320)
    window = dlib.image_window()
    for image in images:
        window.set_image(image)
    im_list = []
    for face in faces:
        im_list.append(dlib.get_face_chip(img, face))
    if len(im_list) != 0:
        for i_, face_ in enumerate(im_list):
            try:
                cv.imwrite(SAVE_CROPPED_FACE_DIR + "/" + str(frame_num) + "_" + str(i_) + ".jpg", cv.resize(face_, (96, 96), interpolation=cv.INTER_AREA))
            except:
                print("cannot save : " + str(frame_num))
                print(face_)
                plt.imshow(face_)
    
SAVE_CROPPED_FACE_DIR = RESULT_DIR + 'Cropped_imgs/' + PROGRAM_START_TIME + '-' + VIDEO_FILE_NAME.split('.')[0]
make_directory(SAVE_CROPPED_FACE_DIR)

VIDEO_FILE_NAME = 'PowerUp.mp4'
m_video_capture = cv.VideoCapture(VIDEO_DIR + VIDEO_FILE_NAME)

CASCADE_DIR = RES_DIR + 'lbpcascade_frontalface_improved.xml'
m_face_cascade = cv.CascadeClassifier(CASCADE_DIR)

TOTAL_FRAME = m_video_capture.get(cv.CAP_PROP_FRAME_COUNT)
FRAME_STEP = 10
LOAD_FAIL = []
if len(os.listdir(SAVE_CROPPED_FACE_DIR)) == 0:
    for frame in tqdm(range(0, int(TOTAL_FRAME), FRAME_STEP)):
        img, frame, ret = get_frame(video_capture=m_video_capture, frame=frame)
        if not ret:
            LOAD_FAIL.append(frame)
            continue
        faces_area = dlib_face_chip(img, frame)
    if len(LOAD_FAIL) > 0:
        print("Fail to load %d of frame(s)" % len(LOAD_FAIL))
        print(LOAD_FAIL)
else:    
    print("Directory is not empty")
    print("Already extracted in %s" % PROGRAM_START_TIME)

Directory is not empty
Already extracted in 20181108


In [18]:
face_list = []
face_files = os.listdir(SAVE_CROPPED_FACE_DIR)
for face_file_name in tqdm(face_files):
    img = cv.imread(SAVE_CROPPED_FACE_DIR + '/' + face_file_name)
#     img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     img_blur = cv.GaussianBlur(img_gray, (3,3), 0)
#     img_sharp = cv.Laplacian(img_blur, cv.CV_64F)
#     corners = cv.cornerHarris(img_gray, 2, 3, 0.04)
#     face_list.append(img_gray)
#     face_list.append(img_sharp.flatten())
#     rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    face_list.append(img)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 981/981 [00:01<00:00, 747.65it/s]


In [33]:
def cluster(face_list, face_files):
    fl2grayflatten = [(cv.cvtColor(img, cv.COLOR_BGR2GRAY)).flatten() for img in face_list]
    encodings = fl2grayflatten
    clt = DBSCAN(metric="euclidean")
    clt.fit(encodings)

    label_ids = np.unique(clt.labels_)
    num_unique_faces = len(np.where(label_ids > -1)[0])
    print(num_unique_faces)
#     for label_id in label_ids:
#         dir_name = "drive/ID%d" % label_id
#         os.mkdir(dir_name)

#         indexes = np.where(clt.labels_ == label_id)[0]

#         for i in  tqdm(indexes):
#             cv.imwrite(dir_name + "/" + str(face_files[i])+ ".jpg", face_list[i])

In [34]:
from sklearn.cluster import DBSCAN
cluster(face_list, face_files)

0
